In [1]:
# ボストンマラソンデータの読み込み

def getBMData(filename):
    """Read CSV File. The elements are as followes.
    0. name, 1. gender, 2. age, 3. division(run or wheelchair), 4. country, 5. time
    return value: dictionary of the 6 value's list
    """
    data = {}
    f = open(filename)
    line = f.readline()
    data['name'], data['gender'], data['age'] = [], [], []
    data['division'], data['country'], data['time'] = [], [], []
    while line != '':
        split = line.split(',')
        data['name'].append(split[0])                                                                                                                                                                                                                 
        data['gender'].append(split[1])
        data['age'].append(split[2])
        data['division'].append(split[3])
        data['country'].append(split[4])
        data['time'].append(float(split[5][:-1])) # remove \n
        line = f.readline()
    f.close()
    return data

In [16]:
# 多重仮説検定

from scipy import stats

data = getBMData('bm_results2012.txt')
countriesToCompare = ['BEL', 'BRA', 'FRA', 'JPN', 'ITA']

countryTimes = {}
for i in range(len(data['name'])):
    if data['country'][i] in countriesToCompare and data['gender'][i] == 'F':
        try:
            countryTimes[data['country'][i]].append(data['time'][i])
        except KeyError:
            countryTimes[data['country'][i]] = [data['time'][i]]

for c1 in countriesToCompare:
    for c2 in countriesToCompare:
        if c1 < c2:
            # equal_var = Falseは2つの母集団の分散が同じか分からないことを示す
            pVal = stats.ttest_ind(countryTimes[c1], countryTimes[c2], equal_var = False)[1]
            if pVal < 0.05:
                print(c1, 'and', c2, 'have significantly different means, p-value =', round(pVal, 4))
                print(c1, 'Mean Time =', sum(countryTimes[c1])/len(countryTimes[c1]))
                print(c2, 'Mean Time=', sum(countryTimes[c2])/len(countryTimes[c2]))

"""
JPNのほうがITAよりも女性ランナーが早いと断定するのは早計だ.
ある国のランナーが他の国よりも平均タイムが早かったのは確かだが, 一方で,
標本サイズがとても小さく, 各国の女性マラソンランナーの能力を代表していないと思われる.

有意水準を5%とすると, 20回に1度は誤って帰無仮説を棄却してしまう(有意であると判断してしまう)可能性がある.

多重仮説を含む実験を行う場合、もっともシンプルで保守的に使われる手法はボンフェローニ補正(Bonferroni correction)である.
その方法は各仮説を1/m*αのレベルで検定することである.
"""
pass

ITA and JPN have significantly different means, p-value = 0.025
ITA Mean Time = 305.35
JPN Mean Time= 265.68812500000007


In [15]:
# p値検定
# 性別による平均タイムの違いの有意性を検証する

from scipy import stats

data = getBMData('bm_results2012.txt')

gengerTimes= {}
for i in range(len(data['name'])):
    try:
        gengerTimes[data['gender'][i]].append(data['time'][i])
    except KeyError:
        gengerTimes[data['gender'][i]] = [data['time'][i]]

for k in gengerTimes:
    print(k, 'Mean Time =', sum(gengerTimes[k])/len(gengerTimes[k]))

print('pvalue =', stats.ttest_ind(gengerTimes['M'], gengerTimes['F'], equal_var = False)[1])

M Mean Time = 254.62222063264872
F Mean Time = 274.88436879116006
pvalue = 1.2505227805509697e-199
